# Lane
Lanes are of two types:
1. Long
2. Short

In general they always have in common a class, the lane-tail, which takes some information and appoints some temporary choices, these choices are candidates, they are chosen by the tail based on an iterator which is then given to the candidate

Once all tails have made their choice the hub collects them all (in the form of lists of strings) and proceeds to iterate over the candidate, if a candidate has more than an offer it accepts one and forwards the others to new candidates.

## Short lanes
These are made up of a single step, meaning they base their decision only on information that is below them, such as totals and the number of elected members in the same area during a previous lane

## Long lanes
Long lanes, unlike short ones, start at a higher level than the GeoEnt tasked with electing the candidates. These upper levels start with a lane-head and pass through various lane-nodes before getting to a lane-tail

The process is as follows:
1. The lane head takes information from lower levels and makes a first decision on a possible distribution
2. It asks the nodes under it to give their "opinion", that is making a distribution ov seats based only on their local knowledge
3. It gathers these opinions and, based on its original decision, adjusts them
4. It passes its decision to the node below

If the node below is the tail then it takes the information and uses it to appoint the appropriate candidates

Else the process starts one level lower

# Hub
The hub in this context has to:
+ Keep track of the lanes and their order of execution
+ Start the lanes in the correct order
+ Iterate over the candidates to get the actual elected ones

# Candidate
The candidate has to:
+ Receive proposals
+ Choose one proposal
+ Propagate other ones

In [1]:
class ExtendableIterator:
    def __init__(self, set_start):
        self.s = set(set_start)
    def add_el(self, el):
        self.s.add(el)
    def __next__(self):
        try:
            return self.s.pop()
        except KeyError:
            raise StopIteration

    def __iter__(self):
        return self

In [2]:
class Hub:
    def __init__(self):
        self.lanes = []
        self.lane_heads = dict()
    
    def start_lanes(self):
        for i in self.lanes:
            self.exec_lane(i)
            
    def get_elector(lane, name):
        """
        Each lane will tell who the electors are for itself, this function returns
        an instance
        """
    
    def add_elected(self, lane_name, tail_name, candidate_name):
        pass
    
    def exec_lane(self, lane):
        head_class = self.lane_heads[lane]
        heads = self.get_instances(head_class) #list
        res_nest = map(lambda h: h.exec_lane(lane), heads) # list of list of four tuples 
                                                           # indicating
                                                           # who must elect how many people
        res_flat = [item for sublist in res_nest for item in sublist]
        set_r = set()
        for elector, amount, lane, district in res_flat:
            elector = self.get_elector(lane, elector)
            
            names = elector.elect(lane, district, amount)
            for i in names:
                set_r.add(i)
        
        it = ExtendableIterator(set_r)
        for i in it: #i: str
            c = self.get_instance("Candidate", i)
            decisione, altri = c.choose(lane) #decisione: str, nome di un'istanza di lane-tail  
                                              #altri: [str]
            for el in altri:
                it.add_el(el)
                
            self.add_elected(lane, decisione, i)

In [ ]:
class Party:
    def elect(self, lane, district, number):
        """
        Asks the coalition (if any) to create an iterator from the point of view of the party for the district.
        Then iterates until it has number candidates considering the proposal
        
        Defined in another metaclass
        """

In [2]:
class Candidate:
    def __init__(self):
        self.options = dict()
        self.elected = False

    def elect(self, lane, district, number):
        """
        Works only for uninominal
        """

    def choose(self, lane):
        """Here is the customizable part of the implementation"""
        chosen = func(self.options)
        del self.options[chosen]
        ret = []
        for proposer,(info, iterat) in self.options.items():
            cont = True
            while cont:
                try:
                    n = next(iterat)
                    r = Hub.get_instance("Candidate", n)
                    if r.get_proposal(lane, proposer, iterator, info):
                        ret.append(n)
                except StopIteration:
                    cont = False
        return chosen, ret
        
    def get_proposal(self, lane, proposer, iterator, info=None):
        if self.elected:
            return False
        else:
            self.options[proposer] = (info, iterator)

# What needs to be specified

## exec_lane

**All lane classes**

Explains how the lane step must be executed.
+ Call point: either the hub calls it on the Lane head or the head or a node calls it on a lower node
+ Arguments: None for the lane head, keyword arguments as needed for lower steps
+ Return value: For the lane tail a series of continuations which when called will return the candidates first nominated. For the nodes above the union of the lower continuations

Further notes on the continuations:

Because of how python handles namespaces it's better to handle it in a different manner than a lambda or generated function, all the information needed is:
+ Who does the electing (the instance of a Class deriving the Elector metaclass)
+ How many seats (an integer)
+ On whose account (the instance)
+ For which lane
This can therefore be represented by a 4-uple, I'll then merge 4-uples in lists

Execution model:
1. Gets called
2. The node creates an ideal seat distribution
3. It asks lower nodes to propose a distribution
4. It adapts the two distributions, obtaining for each lower node a distribution
5. It calls exec_lane on each lower node, passing the relevant information
6. It merges the results and returns

A distribution could be a dictionary object:number of seats, this would be the most intuitive model, however it doesn't allow to pass information downwards, instead I'll make up a distribution to be a more complex data structure

"district_name":(seat_distr, info)
seat_distr :: object:seats_assigned
info :: object:{"info_t":...}

Info will only grow, while seat_distr will not, for instance when dividing a coalition seats first I'll have to divide the seats between the coalitions (generating a seats distribution for the coalition as well as the info) and then I'll use this to divide seats between coalition members, generating a new distribution and new info.

The new distribution will replace the old one, but the information dictionaries will be merged

When passing the info dictionary to a lower level I will add to each child dictionary a pay "class_name":"instance_name"

### Information needed

So, what's needed is:
+ Names of the function that generates 

## log_info

**Only tails**

Called as part of exec_lane before returning, will receive the info dictionary

# Problem
If information to create the rankings is created by the lane tails I'd need to wait for all of them to finish before starting to nominate candidates:

**Solution:** the end of a lane forwards the information and returns the instruction on how to make proposals. Once the hub receives all the instruction it calls them

### propose
Generates the distribution of seats and the information to be passed

Since this might be an iterative process these functions might be more numerous than the lanes.

Function signature: `distribution(self, lane, type, constraints=None, *args, **kwargs)`

The constraint is the higher level distribution, so a `Dict[str,int]`

The return value will be of a distribution (`Dict[str, int]`) and information.

Information is in the form of a dictionary whose keys are either instances or 
references to objects (name, type) and the values a dictionary with keys strings
and values of objects

In [2]:
import pandas as pd
df = pd.DataFrame({'Elettore':['a','b','c','c','c'],'Seats':['a','b','c','d','e'],'Voti':[10,20,5,7,8],'UsedRemainder':[11,20,24,24,24], 'Remainder':[11,20,24,24,24]})

In [17]:
class objTest:
    def __init__(self, df):
        self.df = df
    
    def elettori(self):
        return self.df

    def subs_plurinominali_seggi(self):
        print("subs_seats")
        return 5

o = objTest(df)

def hondt(df, seats):
    print(df, seats)
    return df

# Notes
Distribution, when passed around must be a dataframe with two columns, this allows to use
the existing infrastructure to handle dataframes

In [18]:
import pandas as pd
"""
Example circoscrizione
"""

#def hondt(df, seats):
"""
Df.columns = ['Party', 'Votes']
"""

def propose_coalizione(self, *info, **kwargs):
    seats = self.subs_plurinominali_seggi()
    coalizioni = self.elettori().rename(columns={'Elettore':'Party', 'Voti':'Votes'})
    res = hondt(coalizioni, seats).rename(columns={
        "Party":"Elettore",
        "Votes":"VotiCoalCirco",
        "Seats":"Seggi",
        "Remainder":"RestoCoalCirco",
        "UsedRemainder":"RestoCoalCircoUsato"
    })
    # Elettore has type "PolEnt"
    # key = Elettore: PolEnt
    # distribution = [Voti]
    # info = [RestoCoalCirco, RestoCoalCircoUsato]
    distr = res[['Elettore','Seggi']]
    info = dict()
    for i in res.iterrows():
        #k = Hub.getInstance('PolEnt', i[1]['Elettore'])
        k = i[1]['Elettore']
        info[k] = {c:i[1][c] for c in ['RestoCoalCirco','RestoCoalCircoUsato','VotiCoalCirco']}
    
    return distr, info

def propose_lista(self, *info, constraints, **kwargs)
    """
    As propose_coalizione
    """
    
    
def exec_lane_plurinominale(self, distribution, *info):
    """
    1. Ask for ideal to all the lower levels
    2. Adjust distributions
    3. Generate infos
    4. Propagate
    """
    lower_levels = self.subs_plurinominali()
    proposte = {s:Hub.getInstance('Plurinominale',s).propose_plurinominale() 
                for s in lower_levels}
    correct = adjust_distributions(ideal=distribution, proposals=proposte)
    # correct ora è un dizionario
    # nome_plurinominale : (distribuzione_seggi, info_plurinominale)
    res = [i for coll, (distr, info_spec) 
                for i in Hub.getInstance(
                    'Plurinominale', coll).exec_lane(
                        'plurinominale', 
                        {k: {**v, 'Circoscrizione':self.name} for k, v in info_spec.items()},
                        *info)]
    return res

propose_coalizione(o)

subs_seats
  Party Seats  Votes  UsedRemainder  Remainder
0     a     a     10             11         11
1     b     b     20             20         20
2     c     c      5             24         24
3     c     d      7             24         24
4     c     e      8             24         24 5


(  Elettore Seggi
 0        a     a
 1        b     b
 2        c     c
 3        c     d
 4        c     e,
 {'a': {'RestoCoalCirco': 11, 'RestoCoalCircoUsato': 11},
  'b': {'RestoCoalCirco': 20, 'RestoCoalCircoUsato': 20},
  'c': {'RestoCoalCirco': 24, 'RestoCoalCircoUsato': 24}})

In [ ]:
"""Example for Plurinominale"""

def propose_lista(self, *info, **kwargs):
    seats = self.seggi()
    liste = self.totals("lista", 
                             "eletta").rename(columns={'Partito':'Party', 
                                                       'Voti':'Votes'})
    res = hondt(liste, seats).rename(columns={
        "Party":"Lista",
        "Votes":"VotiListaPluri",
        "Seats":"Seggi",
        "Remainder":"RestoListaPluri",
        "UsedRemainder":"RestoListaPluriUsato"
    })
    # Elettore has type "PolEnt"
    # key = Elettore: PolEnt
    # distribution = [Voti]
    # info = [RestoCoalCirco, RestoCoalCircoUsato]
    distr = res[['Lista','Seggi']]
    info = dict()
    for i in res.iterrows():
        k = Hub.getInstance('PolEnt', i[1]['Lista'])
        info[k] = {c:i[1][c] for c in ['RestoListaPluri',
                                       'RestoListaPluriUsato',
                                       'VotiListaPluri']}
    
    return distr, info

def exec_lane_plurinominale(self, *info, distribution):
    distr_dict = distribution.set_index('Lista')['Seggi'].to_dict()
    ret = [(self, lis, seats) for lis, seats in distr_dict.items()]
    dict_gen = {}
    for dic in info:
        for k, d in dic.items():
            ex_d = dict_gen.get(k, {})
            ex_d.update(d)
            dict_gen[k] =ex_d
    for dest, info in dict_gen:
        info['Plurinominale'] = self.name
        dest.give_info("plurinominale", self, pd.Series(info))
    
    del dict_gen
    return ret